In [1]:
## general
import numpy as np
import pandas as pd
import xarray as xr
import copy
import numpy.ma as ma
from itertools import compress
from sklearn.externals import joblib
import os
import regionmask
import time 
import geopandas


## plotting
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
from matplotlib.lines import Line2D 
import cartopy.crs as ccrs
from matplotlib.colors import LogNorm
import mplotutils as mpu

## statistics
from statsmodels.nonparametric.smoothers_lowess import lowess # lowess filter
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

# extra stuff for Lea's loading codes
import glob
import datetime



/home/tristan/miniconda3/envs/mesmer-env/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/tristan/miniconda3/envs/mesmer-env/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
/home/tristan/miniconda3/envs/mesmer-env/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guid

'0.9.0'

In [11]:
## my stuff
os.chdir('/home/tristan/mesmer/tools/')
from loading import load_data_obs,load_data_single_mod
os.chdir('/home/tristan/mesmer/plots/')

In [6]:
GMST_idx = True
compute_linreg=False
ref_start='1951-01-01'
ref_end='1981-01-01'
if GMST_idx == True:
    dir_in_data= '/home/tristan/mesmer/data/'
    dir_out_plot = '/home/tristan/mesmer/plots/'

## code from Lea

In [2]:
def norm_cos_wgt(lats):
    
    from numpy import cos, deg2rad
    
    weights = np.cos(np.deg2rad(lats))
    
    return weights

In [3]:
def load_constant_files():
    """ Load all the constant files

        Keyword argument:
        - None

        Output:
        - srex: srex index on land
        - srex_names: the names of the srex regions
        - df_srex: data frame containing the shape files of the srex regions
        - srex_raw: gridded srex dataset including lat/lon
        - lon_pc: longitudes for pcolormesh (needs 1 more than on grid)
        - lat_pc: latitudes for pcolormesh (needs 1 more than on grid)
        - idx_l: array with 0 where sea, 1 where land (assumption: land if frac land > 0)
        - wgt_l: land grid point weights to derive area weighted mean vars
        - wgt: weights of each grid point to derive area weighted mean vars
        - frac_l_raw: array with actual fractions of land / sea according to mask (0 all ocean, 1 all land)
     """   
     # load in the constants files 
    
    dir_data = '/home/tristan/mesmer/data/'
    dir_srex_shape='/referenceRegions/referenceRegions.shp'
    file_ls = 'interim_invariant_lsmask_regrid.nc'
    file_srex_shape = 'referenceRegions.shp'

    ## SREX stuff
    df_all_regs = geopandas.read_file(dir_data+file_srex_shape)
    srex_names = ['ALA','CGI','WNA','CNA','ENA','CAM','AMZ','NEB','WSA','SSA','NEU','CEU','MED','SAH','WAF','EAF','SAF',
             'NAS','WAS','CAS','TIB','EAS','SAS','SEA','NAU','SAU'] # SREX names ordered according to SREX mask I am 
                    # employing
    df_srex = df_all_regs.loc[df_all_regs['LAB'].isin(srex_names)] # alternative indexing: search in column LAB for names
    srex_raw = xr.open_mfdataset(dir_data+file_srex) # srex_raw nrs from 1-26
    lons, lats = np.meshgrid(srex_raw.lon.values,srex_raw.lat.values) # the lon, lat grid (just to derive weights)    

    ## land mask
    frac_l = xr.open_mfdataset(dir_data+file_ls) #land-sea mask of ERA-interim bilinearily interpolated 
    frac_l_raw = np.squeeze(copy.deepcopy(frac_l.lsm.values)) # important to squeeze because use as template for others later on
    frac_l = frac_l.where(frac_l.lat>-60,0) # remove Antarctica from frac_l field (ie set frac l to 0)
    idx_l=np.squeeze(frac_l.lsm.values)>0.0 # idex land #-> everything >0 I consider land

    wgt = norm_cos_wgt(lats) # area weights of each grid point
    wgt_l = (wgt*frac_l_raw)[idx_l] # area weights for land grid points (including taking fraction land into consideration)
    lon_pc, lat_pc = mpu.infer_interval_breaks(frac_l.lon, frac_l.lat) # the lon / lat for the plotting with pcolormesh
    srex=(np.squeeze(srex_raw.srex_mask.values)-1)[idx_l] # srex indices on land
    
    
    if (srex_raw.lon!=frac_l.lon).any():
        print('There is an error. The srex_raw and the frac_l grids do not agree.') 
        
    return srex,srex_names,df_srex,srex_raw,lon_pc,lat_pc,idx_l,wgt_l,wgt,frac_l_raw

In [8]:
def compute_Tan_wgt_reg(gen,reg,obs_idx=False):
    print(gen,reg)

    if reg == 'Global_land':
        idx_reg_l=np.ones(3043,dtype=bool)
        idx_reg_l_grid = copy.deepcopy(idx_l)
        
    else:   
        idx_reg = srex_names.index(reg) # index region
        idx_reg_l = (srex==idx_reg) #index land (l) gp inside specific srex region (reg)
        srex_grid=np.zeros(idx_l.shape)
        srex_grid[idx_l]=srex # ATTENTION: not 100% convinced yet if this is correct (I get some missing values that
        # I am not sure I used to get earlier too. Should investigate once I have time)
        idx_reg_l_grid = (srex_grid==idx_reg)#& idx_l 
 
    Tan_wgt_reg_y={}


    if gen == 5:
        models_sel = models_cmip5
    elif gen == 6:
        models_sel = models
        
    for model in models_sel:
        Tan_wgt_reg_y[model]={}

        for run_nr in y[model].keys():
            Tan_wgt_reg_y[model][run_nr] = np.zeros(len(time[model]))

            for i in np.arange(len(time[model])):
                Tan_wgt_reg_y[model][run_nr][i] = np.average(y[model][run_nr][i][idx_reg_l],weights=wgt_l[idx_reg_l]) 
                
    if obs_idx == True:
        obs_Tan_wgt_reg_y={}
        
        for model in obs:
            obs_Tan_wgt_reg_y[model]=np.zeros(len(obs_time[model]))
            for i in np.arange(len(obs_time[model])):
                idx_valid=~np.isnan(obs_y[model][i][idx_reg_l])
                obs_Tan_wgt_reg_y[model][i] = np.average(obs_y[model][i][idx_reg_l][idx_valid],weights=wgt_l[idx_reg_l][idx_valid]) 
            
    if obs_idx == False:
        return Tan_wgt_reg_y,idx_reg_l_grid
    else:
        return Tan_wgt_reg_y,idx_reg_l_grid,obs_Tan_wgt_reg_y

In [9]:
reg='Global_land'
Tan_wgt_reg_y,idx_reg_l_grid_gl,obs_Tan_wgt_reg_y = compute_Tan_wgt_reg(6,reg,obs_idx=True)

6 Global_land


NameError: name 'idx_l' is not defined

In [1]:
models = ['ACCESS-CM2','ACCESS-ESM1-5','BCC-CSM2-MR','CAMS-CSM1-0','CanESM5','CESM2-WACCM',
          'CNRM-CM6-1','CNRM-CM6-1-HR','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','FIO-ESM-2-0','GFDL-CM4',
          'GFDL-ESM4','HadGEM3-GC31-LL','INM-CM4-8','INM-CM5-0','IPSL-CM6A-LR','MIROC6','MIROC-ES2L','MPI-ESM1-2-HR',
          'MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorESM2-LM','NorESM2-MM','UKESM1-0-LL']

print(len(models))

27


In [ ]:
obs_Tglob={} # ATTENTION: this is a BLENDED product with SST over ocean and tas over land + sea ice
obs_time={}
obs_y={}  # ATTENTION: this is a BLENDED product with SST over ocean and tas over land + sea ice
obs_y['cowtan'],obs_time['cowtan'],obs_Tglob['cowtan']=load_data_obs('cowtan',Tref_start=ref_start_obs,Tref_end=ref_end_obs,Tblendglob_idx=True)

obs=['best']
    
obs_idx_t_start_glob = np.where(obs_time['best']==start_year_glob)[0][0]
obs_idx_t_start_reg = np.where(obs_time['best']==start_year_reg)[0][0]

In [ ]:
y={}
GMST={}
GSAT={}
time={} # because CAMS-CSM1 has 1 less entry than all other models
for model in models:
    y[model],time[model],srex,srex_names,df_srex,lon_pc,lat_pc,idx_l,wgt_l,GSAT[model],GMST[model]=load_data_single_mod(6,model,'ssp585',GMST_idx=True,Tref_all=True,Tref_start=ref_start,Tref_end=ref_end)


for model in models_cmip5:    
    y[model],time[model],srex,srex_names,df_srex,lon_pc,lat_pc,idx_l,wgt_l,GSAT[model]=load_data_single_mod(5,model,'rcp85',GMST_idx=False,Tref_all=True,Tref_start=ref_start,Tref_end=ref_end)

    
Tanglob_wgt={}
for model in models:
    Tanglob_wgt[model]={}
    for run in y[model].keys():
        if GMST_idx==True:
            Tanglob_wgt[model][run]=GMST[model][run]
        else:
            Tanglob_wgt[model][run]=GSAT[model][run]

In [5]:
#srex
v_srex=np.array([-0.5,0.5,1.5]) # to mark which SREX region I am in
cmap_srex, norm_srex = from_levels_and_colors(v_srex, colors=['white','#31a354'], extend='neither') # #d95f02

45


In [ ]:
fig=plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 10})

grid = plt.GridSpec(1, 3, wspace=0.05, hspace=0.) # create a grid for the subplots

y_min=-3.5
y_max=13.5

ax1 = plt.subplot(grid[0,0])
reg='Global_land'
Tan_wgt_reg_y,idx_reg_l_grid_gl,obs_Tan_wgt_reg_y = compute_Tan_wgt_reg(6,reg,obs_idx=True)
    
for model in models:
    run=next(iter(y[model]))
    plt.plot(time[model],Tan_wgt_reg_y[model][run],color='#377eb8',lw=0.5,alpha=0.6)

plt.plot(obs_time['best'][obs_idx_t_start_reg:],obs_Tan_wgt_reg_y['best'][obs_idx_t_start_reg:],color='k',lw=0.5)
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.xlim([time[models[0]][0],time[models[0]][-1]])
plt.ylim([y_min,y_max])
plt.yticks(np.arange(-3,12.1,3))
plt.xticks(np.arange(1900,2100,60))

plt.xlabel('Year [-]')
plt.ylabel('T [°C]')
if reg == 'Global_land':
    plt.title('Global Land')


custom_lines = [Line2D([0], [0], color='#377eb8',alpha=0.7),
                Line2D([0], [0],color='#e41a1c',alpha=0.7),
                Line2D([0], [0],color='k')] 
plt.legend(custom_lines, ['CMIP6 (27)','CMIP5 (40)','observations'],loc=2)




ax1 = plt.subplot(grid[0,1])
reg='ENA'#'CEU'#ENA
Tan_wgt_reg_y,idx_reg_l_grid,obs_Tan_wgt_reg_y = compute_Tan_wgt_reg(6,reg,obs_idx=True)
Tan_wgt_reg_y_cmip5,idx_reg_l_grid=compute_Tan_wgt_reg(5,reg)
    
for model in models:
    run=next(iter(y[model]))
    plt.plot(time[model],Tan_wgt_reg_y[model][run],color='#377eb8',lw=0.5,alpha=0.6)

plt.plot(obs_time['best'][obs_idx_t_start_reg:],obs_Tan_wgt_reg_y['best'][obs_idx_t_start_reg:],color='k',lw=0.5)
plt.xlim([time[models[0]][0],time[models[0]][-1]])

plt.xlabel('Year [-]')
plt.gca().yaxis.set_major_formatter(plt.NullFormatter())
if reg == 'ENA':
    plt.title('East North America')
plt.ylim([y_min,y_max])
plt.yticks(np.arange(-3,12.1,3))
plt.xticks(np.arange(1900,2100,60))

ax_sub = fig.add_axes([0.348, 0.65, 0.2, 0.2],projection=ccrs.Robinson(central_longitude=0))
ax_sub.add_geometries(df_srex.geometry.values, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='k',lw=0.5)
ax_sub.coastlines(lw=0.5)
mesh_srex = ax_sub.pcolormesh(lon_pc, lat_pc, idx_reg_l_grid, cmap=cmap_srex,norm=norm_srex,transform=ccrs.PlateCarree(),rasterized=True)


ax1 = plt.subplot(grid[0,2])
reg='EAF'#'SSA'#SAF
Tan_wgt_reg_y,idx_reg_l_grid,obs_Tan_wgt_reg_y = compute_Tan_wgt_reg(6,reg,obs_idx=True)
Tan_wgt_reg_y_cmip5,idx_reg_l_grid=compute_Tan_wgt_reg(5,reg)
    
for model in models:
    run=next(iter(y[model]))
    plt.plot(time[model],Tan_wgt_reg_y[model][run],color='#377eb8',lw=0.5,alpha=0.6)

plt.plot(obs_time['best'][obs_idx_t_start_reg:],obs_Tan_wgt_reg_y['best'][obs_idx_t_start_reg:],color='k',lw=0.5)
plt.xlim([time[models[0]][0],time[models[0]][-1]])

plt.xlabel('Year [-]')
#plt.ylabel('T [°C]')
plt.gca().yaxis.set_major_formatter(plt.NullFormatter())
if reg == 'EAF':
    plt.title('East Africa')
plt.ylim([y_min,y_max])
plt.yticks(np.arange(-3,12.1,3))
plt.xticks(np.arange(1900,2100,60))

ax_sub = fig.add_axes([0.608, 0.65, 0.2, 0.2],projection=ccrs.Robinson(central_longitude=0))
ax_sub.add_geometries(df_srex.geometry.values, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='k',lw=0.5)
ax_sub.coastlines(lw=0.5)
mesh_srex = ax_sub.pcolormesh(lon_pc, lat_pc, idx_reg_l_grid, cmap=cmap_srex,norm=norm_srex,transform=ccrs.PlateCarree(),rasterized=True)



ax_sub = fig.add_axes([0.205, 0.65, 0.2, 0.2],projection=ccrs.Robinson(central_longitude=0))
ax_sub.add_geometries(df_srex.geometry.values, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='k',lw=0.5)
ax_sub.coastlines(lw=0.5)
mesh_srex = ax_sub.pcolormesh(lon_pc, lat_pc, idx_reg_l_grid_gl, cmap=cmap_srex,norm=norm_srex,transform=ccrs.PlateCarree(),rasterized=True)


plt.savefig(dir_out_plot+'ts_Global_land_ENA_EAF_cmip6_obs_'+str(start_year_reg)+'.png' ,dpi=200 ,bbox_inches='tight')
plt.savefig(dir_out_plot+'ts_Global_land_ENA_EAF_cmip6_obs_'+str(start_year_reg)+'.pdf' ,dpi=200 ,bbox_inches='tight')
